## Joins

In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day3").getOrCreate()


23/09/03 19:05:55 WARN Utils: Your hostname, kushal-Latitude-E5440 resolves to a loopback address: 127.0.1.1; using 192.168.1.14 instead (on interface wlp2s0)
23/09/03 19:05:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/03 19:05:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/03 19:05:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.



Question: You are given two DataFrames: employees_df and departments_df, which contain information about employees and their respective departments. The schema for the DataFrames is as follows:

employees_df schema:
|-- employee_id: integer (nullable = true)
|-- employee_name: string (nullable = true)
|-- department_id: integer (nullable = true)

departments_df schema:

|-- department_id: integer (nullable = true)
|-- department_name: string (nullable = true)

Employees DataFrame:
                                                                                
+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|1          |Pallavi mam  |101          |
|2          |Bob          |102          |
|3          |Cathy        |101          |
|4          |David        |103          |
|5          |Amrit Sir    |104          |
|6          |Alice        |null         |
|7          |Eva          |null         |
|8          |Frank        |110          |
|9          |Grace        |109          |
|10         |Henry        |null         |
+-----------+-------------+-------------+



Departments DataFrame:
+-------------+------------------------+
|department_id|department_name         |
+-------------+------------------------+
|101          |HR                      |
|102          |Engineering             |
|103          |Finance                 |
|104          |Marketing               |
|105          |Operations              |
|106          |null                    |
|107          |Operations              |
|108          |Production              |
|null         |Finance                 |
|110          |Research and Development|
+-------------+----------------------


In [2]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [6]:
employees_schema = StructType([
    StructField("employee_id", IntegerType(), True),
    StructField("employee_name", StringType(), True),
    StructField("department_id", IntegerType(), True)
])

departments_schema = StructType([
    StructField("department_id", IntegerType(), True),
    StructField("department_name", StringType(), True)
])

In [7]:
employees_data = [
    (1, "Pallavi mam", 101),
    (2, "Bob", 102),
    (3, "Cathy", 101),
    (4, "David", 103),
    (5, "Amrit Sir", 104),
    (6, "Alice", None),
    (7, "Eva", None),
    (8, "Frank", 110),
    (9, "Grace", 109),
    (10, "Henry", None)
]

departments_data = [
    (101, "HR"),
    (102, "Engineering"),
    (103, "Finance"),
    (104, "Marketing"),
    (105, "Operations"),
    (106, None),
    (107, "Operations"),
    (108, "Production"),
    (None, "Finance"),
    (110, "Research and Development")
]

In [8]:
employees_df = spark.createDataFrame(employees_data, schema=employees_schema)
departments_df = spark.createDataFrame(departments_data, schema=departments_schema)


In [10]:
employees_df.show()
employees_df.printSchema()
departments_df.show()
departments_df.printSchema()

+-----------+-------------+-------------+
|employee_id|employee_name|department_id|
+-----------+-------------+-------------+
|          1|  Pallavi mam|          101|
|          2|          Bob|          102|
|          3|        Cathy|          101|
|          4|        David|          103|
|          5|    Amrit Sir|          104|
|          6|        Alice|         null|
|          7|          Eva|         null|
|          8|        Frank|          110|
|          9|        Grace|          109|
|         10|        Henry|         null|
+-----------+-------------+-------------+

root
 |-- employee_id: integer (nullable = true)
 |-- employee_name: string (nullable = true)
 |-- department_id: integer (nullable = true)

+-------------+--------------------+
|department_id|     department_name|
+-------------+--------------------+
|          101|                  HR|
|          102|         Engineering|
|          103|             Finance|
|          104|           Marketing|
|          

In [43]:
departments_df.createOrReplaceTempView("department")
employees_df.createOrReplaceTempView("employee")

### Join Expressions

Question: How can you combine the employees_df and departments_df DataFrames based on the common "department_id" column to get a combined DataFrame with employee names and their respective department names?

In [14]:
#basic syntax of join is 'df1.join(df2, joinExpression, joinType)
#here joinexpression is the common column and jointype is the type of join to be performed
employees_df.join(departments_df, "department_id", "inner").show(truncate = False)


+-------------+-----------+-------------+------------------------+
|department_id|employee_id|employee_name|department_name         |
+-------------+-----------+-------------+------------------------+
|101          |1          |Pallavi mam  |HR                      |
|101          |3          |Cathy        |HR                      |
|102          |2          |Bob          |Engineering             |
|103          |4          |David        |Finance                 |
|104          |5          |Amrit Sir    |Marketing               |
|110          |8          |Frank        |Research and Development|
+-------------+-----------+-------------+------------------------+



In [48]:
p1 = spark.sql("""
                select  e.department_id, e.employee_id, e.employee_name, d.department_name 
                from employee e
               join department d
               on e.department_id = d.department_id

""")
p1.show()

+-------------+-----------+-------------+--------------------+
|department_id|employee_id|employee_name|     department_name|
+-------------+-----------+-------------+--------------------+
|          101|          1|  Pallavi mam|                  HR|
|          101|          3|        Cathy|                  HR|
|          102|          2|          Bob|         Engineering|
|          103|          4|        David|             Finance|
|          104|          5|    Amrit Sir|           Marketing|
|          110|          8|        Frank|Research and Deve...|
+-------------+-----------+-------------+--------------------+



### Inner Joins

Question: How can you retrieve employee names and their respective department names for employees belonging to the "Engineering" department?

In [19]:
ijj = employees_df.join(departments_df, "department_id", "inner")
ij = ijj.select("employee_name", "department_name")
ij.where(ij.department_name == "Engineering").show()

+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
|          Bob|    Engineering|
+-------------+---------------+



In [51]:
p2 = spark.sql("""
                select  e.employee_name, d.department_name 
                from employee e
               join department d
               on e.department_id = d.department_id
               where d.department_name = "Engineering"

""")
p2.show()

+-------------+---------------+
|employee_name|department_name|
+-------------+---------------+
|          Bob|    Engineering|
+-------------+---------------+



### Outer Joins

Question: Retrieve a DataFrame that contains all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [23]:
ojj = employees_df.join(departments_df, "department_id", "outer")
oj = ojj.select("employee_name", "department_name")
#oj.show()
filled_oj = oj.fillna("No Department", subset=["department_name"])
filled_oj.show() #according to question
actual_oj = oj.fillna("No Department")
actual_oj.show() #according to expected output

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|        Alice|       No Department|
|          Eva|       No Department|
|        Henry|       No Department|
|         null|             Finance|
|  Pallavi mam|                  HR|
|        Cathy|                  HR|
|          Bob|         Engineering|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|         null|          Operations|
|         null|       No Department|
|         null|          Operations|
|         null|          Production|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|        Alice|       No Department|
|          Eva|       No Department|
|        Henry|       No Department|
|No Department|             Finance|
|  Pallavi mam|                  HR|
|        Cathy|                  HR|
|          Bob|         Engineering|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|No Department|          Operations|
|No Department|       No Department|
|No Department|          Operations|
|No Department|          Production|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



In [57]:
p3 = spark.sql("""
                select  coalesce(e.employee_name, "No Department") as employee_name,
                        coalesce(d.department_name, "No Department") as department_name
                from employee e
               full outer join department d
               on e.department_id = d.department_id
""")
p3.show()

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|        Alice|       No Department|
|          Eva|       No Department|
|        Henry|       No Department|
|No Department|             Finance|
|  Pallavi mam|                  HR|
|        Cathy|                  HR|
|          Bob|         Engineering|
|        David|             Finance|
|    Amrit Sir|           Marketing|
|No Department|          Operations|
|No Department|       No Department|
|No Department|          Operations|
|No Department|          Production|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



### Left Outer Joins

Question: List all employees along with their department names. If an employee doesn't have a department assigned, display "No Department".

In [25]:
lojj = employees_df.join(departments_df, "department_id", "left_outer")
loj = lojj.select("employee_name", "department_name")
loj.fillna("No Department").show()

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|  Pallavi mam|                  HR|
|          Bob|         Engineering|
|        Cathy|                  HR|
|        David|             Finance|
|        Alice|       No Department|
|    Amrit Sir|           Marketing|
|          Eva|       No Department|
|        Henry|       No Department|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



In [58]:
p4 = spark.sql("""
                select  e.employee_name,
                        coalesce(d.department_name, "No Department") as department_name
                from employee e
               left outer join department d
               on e.department_id = d.department_id
""")
p4.show()

+-------------+--------------------+
|employee_name|     department_name|
+-------------+--------------------+
|  Pallavi mam|                  HR|
|          Bob|         Engineering|
|        Cathy|                  HR|
|        David|             Finance|
|        Alice|       No Department|
|    Amrit Sir|           Marketing|
|          Eva|       No Department|
|        Henry|       No Department|
|        Grace|       No Department|
|        Frank|Research and Deve...|
+-------------+--------------------+



### Right Outer Joins

Question: Display a list of departments along with employee names. If a department has no employees, display "No Employees".



In [33]:
rojj = employees_df.join(departments_df, "department_id", "right_outer")
roj = rojj.select("department_name", "employee_name")
roj.fillna("No Employee").show()

+--------------------+-------------+
|     department_name|employee_name|
+--------------------+-------------+
|                  HR|        Cathy|
|                  HR|  Pallavi mam|
|         Engineering|          Bob|
|             Finance|        David|
|           Marketing|    Amrit Sir|
|          Operations|  No Employee|
|         No Employee|  No Employee|
|          Production|  No Employee|
|             Finance|  No Employee|
|          Operations|  No Employee|
|Research and Deve...|        Frank|
+--------------------+-------------+



In [60]:
p5 = spark.sql("""
                select coalesce(d.department_name, "No Employee") as department_name, 
                        coalesce(e.employee_name, "No Employee") as employee_name
                from employee e
               right outer join department d
               on e.department_id = d.department_id
""")
p5.show()

+--------------------+-------------+
|     department_name|employee_name|
+--------------------+-------------+
|                  HR|        Cathy|
|                  HR|  Pallavi mam|
|         Engineering|          Bob|
|             Finance|        David|
|           Marketing|    Amrit Sir|
|          Operations|  No Employee|
|         No Employee|  No Employee|
|          Production|  No Employee|
|             Finance|  No Employee|
|          Operations|  No Employee|
|Research and Deve...|        Frank|
+--------------------+-------------+



### Left Semi Joins

Question: Retrieve a DataFrame that includes employee names for departments that have employees.



In [32]:
lsjj = employees_df.join(departments_df, "department_id", "left_semi")
lsj = lsjj.select("employee_name")
lsj.show()

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Frank|
+-------------+



In [61]:
p6 = spark.sql("""
                select  e.employee_name 
                from employee e
               left semi join department d
               on e.department_id = d.department_id

""")
p6.show()

+-------------+
|employee_name|
+-------------+
|  Pallavi mam|
|        Cathy|
|          Bob|
|        David|
|    Amrit Sir|
|        Frank|
+-------------+



### Left Anti Joins

Question: Find the employees who don't belong to any department.

In [34]:
lajj = employees_df.join(departments_df, "department_id", "left_anti")
laj = lajj.select("employee_name")
laj.show()

+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Henry|
|        Grace|
+-------------+



In [62]:
p7 = spark.sql("""
                select e.employee_name 
                from employee e
               left anti join department d
               on e.department_id = d.department_id

""")
p7.show()

+-------------+
|employee_name|
+-------------+
|        Alice|
|          Eva|
|        Henry|
|        Grace|
+-------------+



### Cross (Cartesian) Joins

Question: Create a DataFrame that contains all possible combinations of employees and departments.

In [39]:
#employees_df.join(departments_df, "department_id", "cross")
employees_df.crossJoin(departments_df).show()

+-----------+-------------+-------------+-------------+--------------------+
|employee_id|employee_name|department_id|department_id|     department_name|
+-----------+-------------+-------------+-------------+--------------------+
|          1|  Pallavi mam|          101|          101|                  HR|
|          1|  Pallavi mam|          101|          102|         Engineering|
|          2|          Bob|          102|          101|                  HR|
|          2|          Bob|          102|          102|         Engineering|
|          1|  Pallavi mam|          101|          103|             Finance|
|          1|  Pallavi mam|          101|          104|           Marketing|
|          2|          Bob|          102|          103|             Finance|
|          2|          Bob|          102|          104|           Marketing|
|          1|  Pallavi mam|          101|          105|          Operations|
|          1|  Pallavi mam|          101|          106|                null|

In [63]:
p8 = spark.sql("""
                select * 
                from employee e
               cross join department d
""")
p8.show()

+-----------+-------------+-------------+-------------+--------------------+
|employee_id|employee_name|department_id|department_id|     department_name|
+-----------+-------------+-------------+-------------+--------------------+
|          1|  Pallavi mam|          101|          101|                  HR|
|          1|  Pallavi mam|          101|          102|         Engineering|
|          2|          Bob|          102|          101|                  HR|
|          2|          Bob|          102|          102|         Engineering|
|          1|  Pallavi mam|          101|          103|             Finance|
|          1|  Pallavi mam|          101|          104|           Marketing|
|          2|          Bob|          102|          103|             Finance|
|          2|          Bob|          102|          104|           Marketing|
|          1|  Pallavi mam|          101|          105|          Operations|
|          1|  Pallavi mam|          101|          106|                null|